**5. Interactive Dashboard: Analyzing Launches with Dash and Plotly 📊**

This notebook implements the fifth phase of the project: creating a dynamic, interactive web dashboard using the Dash framework and Plotly visualization library. The dashboard serves as a crucial Business Intelligence (BI) tool for non-technical stakeholders (e.g., Space Y management) to visually explore launch data and confirm analytical hypotheses before final machine learning prediction.

**Objective**

The core objective is to provide a user-friendly interface that allows dynamic filtering of SpaceX launch records based on Launch Site and Payload Mass, immediately updating two key visualizations to reveal performance trends.

**💾 Phase 1: Data Setup and Application Layout**

The initial code loads the required libraries (pandas, dash, plotly.express) and defines the entire user interface (UI) structure of the application.

**Key Layout Components:**

Title: SpaceX Launch Records Dash

Launch Site Dropdown (site-dropdown): Allows selecting a single site (CCAFS LC-40, VAFB SLC-4E, KSC LC-39A, CCAFS SLC-40) or viewing 'ALL Sites'. This is the first critical input for analysis.

Payload Mass Range Slider (payload-slider): Allows users to filter missions based on a customizable range of Payload Mass (kg). This tests the hypothesis that payload limits affect reusability success.

Pie Chart (success-pie-chart): Visual output showing the success rate (1 vs 0) for the selected launch site.

Scatter Plot (success-payload-scatter-chart): Visual output comparing Payload Mass (kg) against the class outcome, colored by Booster Version Category to highlight technological evolution.

⚙️ Phase 2: Defining Interactive Callbacks (The Engine)
Dash dashboards operate using callbacks—functions that automatically run when an Input component's value changes, updating an Output component.

1. Pie Chart Callback (get_pie_chart)
Input: Value from the site-dropdown.

Logic:

If 'ALL' is selected, it generates a Pie Chart showing the total success count across all sites, grouped by Launch Site.

If a specific site is selected, it filters the data and shows the success/failure breakdown (class 1 vs 0) for that site only.

Output: Updates the success-pie-chart figure.

2. Scatter Plot Callback (scatter)
Input: Values from both the site-dropdown AND the payload-slider.

Logic:

The data is first filtered to match the selected payload range.

If 'ALL' is selected, it plots the filtered data for all sites.

If a specific site is selected, it filters the data further to plot only that site's missions.

The chart uses Booster Version Category for color coding, visually connecting payload mass to specific hardware generations and their success rates.

Output: Updates the success-payload-scatter-chart figure.

✅ Conclusion: Analytical Value
The generated interactive dashboard is a powerful analytical tool that completes the visualization phase of the project. It allows for immediate visual confirmation of the following business insights:

Site Performance: Quickly compare which launch site yields the highest success percentage.

Payload Sensitivity: Visually identify the range of payload masses that consistently lead to successful or failed landings, providing critical input for mission risk assessment.

These visual findings are essential for validating the upcoming Machine Learning model and guiding data-driven launch decisions for Space Y.

In [1]:
# Install Dash library for building interactive web applications and dashboards
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 36.3 MB/s eta 0:00:00


In [3]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py"

--2025-09-26 19:47:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.42K  --.-KB/s    in 0s      

2025-09-26 19:47:45 (683 MB/s) - ‘spacex_launch_dash.csv’ saved [2476/2476]

--2025-09-26 19:47:45--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/t4-Vy4iOU19i8y6E3Px_ww/spacex-dash-app.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.clo

In [4]:
# Import required libraries
import pandas as pd  # Data manipulation and analysis
import dash  # Dash framework for interactive web apps
from dash import dcc  # Dash Core Components for interactive elements
from dash import html  # Dash HTML components for layout
from dash.dependencies import Input, Output  # For linking components
import plotly.express as px  # High-level Plotly API for plotting

# Read SpaceX launch data into a pandas DataFrame
spacex_df = pd.read_csv("spacex_launch_dash.csv")

# Determine the maximum and minimum payload mass for slider
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Initialize Dash application
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    # Add a title
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Dropdown component to select Launch Site
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',  # Default value
        placeholder='Select a Launch Site here',
        searchable=True
    ),

    html.Br(),

    # Pie chart to show total successful launches
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Payload range slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        value=[min_payload, max_payload]  # Initial selected range
    ),

    # Scatter plot for payload vs. success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback to update pie chart based on selected launch site
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    # Filter dataframe based on dropdown selection
    filtered_df = spacex_df
    if entered_site == 'ALL':
        # Pie chart for all launch sites
        fig = px.pie(
            filtered_df,
            values='class',
            names='Launch Site',
            title='Success Count for All Launch Sites'
        )
    else:
        # Pie chart for a specific launch site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(
            filtered_df,
            values='class count',
            names='class',
            title=f"Total Success Launches for site {entered_site}"
        )
    return fig

# Callback to update scatter chart based on dropdown and slider
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def scatter(entered_site, payload):
    # Filter data based on payload mass range
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0], payload[1])]

    if entered_site == 'ALL':
        # Scatter plot for all sites
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title='Success count vs. Payload Mass for all sites'
        )
    else:
        # Scatter plot for a specific site
        filtered_df_site = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(
            filtered_df_site,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f"Success count vs. Payload Mass for site {entered_site}"
        )
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run()

<IPython.core.display.Javascript object>